Image Binary Classification 

In [ ]:
# importing packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy
import matplotlib.pyplot as plt
import h5py

In [ ]:
# creating dataset with dataset class
class H5Data(Dataset):
    def __init__(self, h5_file, transform=None):
        self.transform = transform
        with h5py.File(h5_file, 'r') as f:
            self.data = f['train_set_x'][:] 
            self.labels = f['train_set_y'][:] 

        self.label_mapping = {'cat': 0, 'non-cat': 1}

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, id):
        image = self.data[id]
        label = self.labels[id]

        if isinstance(label, bytes):
            label = label.decode('utf-8')  
            label = self.label_mapping[label]

        if self.transform:
            image = self.transform(image)

        # Converting images to tensors
        image = torch.tensor(image, dtype=torch.float32) 
        label = torch.tensor(label, dtype=torch.long)  

        return image, label

transform = transforms.Compose([
    transforms.ToPILImage(),               
    transforms.Resize((64, 64)),          
    transforms.ToTensor(),           
    transforms.Normalize(mean=[0.5], std=[0.5])  #
])

# Loading in dataset
cat_data = H5Data("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Project 6/train_catvnoncat.h5", transform = transform)
cat_data_loader = DataLoader(cat_data, batch_size=32, shuffle=True)

In [ ]:
# Creating deep neural network 
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# setting model based on image parameters
input_size = 64 * 64 *3
hidden_size = 5
output_size = 2
m1 = DeepNN(input_size, hidden_size, output_size)

# setting loss function with cross entropy loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(m1.parameters(), lr=0.01)

epochs = 100
for epoch in range(epochs):
    for images, labels in cat_data_loader:
        optimizer.zero_grad()
        outputs = m1(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()